In [356]:
import import_ipynb

import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
import json
from data_generator import DataLoaderSequence
import math
import random
from PyEMD import EMD

In [357]:
class DataLoader():
    """A class for loading and transforming data for the EMD_lstm model"""

    def __init__(self, filename, split1, split2, cols, pre_len, input_timesteps,seq_len,**EMD_para):
        """
        :param filename: the name of the file contains the data, type: .csv
        :param split1: split the data into 2 parts: training, (validation, test)
        :param split2: split the data into 2 parts: validation, test
        |-------------------------------------------|-------------|--------------|
                                                 split1(0.7)   split2(0.85)     
        :param cols: the features 
        :param pre_len: the prediction length (24, 48....)
        :param input_timesteps: the length of looking back (1 month or 1 year), unit: hours
        :param seq_len: the sum of input_timesteps and pre_len
        :param **EMD_para: if apply EMD before LSTM
        """
        
        self.dataframe = pd.read_csv(filename,sep=',')
        if not isinstance(self.dataframe.index, pd.DatetimeIndex):
            self.dataframe['Date_Time'] = pd.to_datetime(self.dataframe['Date_Time'])
            self.dataframe = self.dataframe.set_index('Date_Time')
        
        self.cols = cols
        self.split1 = split1
        self.split2 = split2
        self.len_train_windows = None
        self.pre_len = pre_len
        self.input_timesteps = input_timesteps
        self.seq_len = seq_len
        print('the input cols are:', self.cols)
        self.Normalization(**EMD_para)
    
    def scale_EMD(self, activate_EMD=False):
        '''
        using log to scale the data in order to emphasize the peak
        
        param activate_EMD: decide whether use EMD to do preprocessing
        '''
        for col in self.cols:
            if col == 'Consumption':
                self.dataframe['Consumption'] = self.dataframe.set_index('Consumption').index.map(lambda x: math.log(x))
                print('scaling Consumption is done!')
        
        if activate_EMD==True:

            self.IMFs = EMD().emd(self.dataframe['Consumption'].values)
            print('the signal is decomposed into '+ str(self.IMFs.shape[0]) +' parts')
            
            self.df_names_IMF = locals()
            
            for ind, IMF in enumerate(self.IMFs):
                
                IMF_name = 'IMF'+str(ind)+'_consumption'
                data={IMF_name:self.IMFs[ind]}
                IMF_i = pd.DataFrame(data=data)
                self.df_names_IMF['IMF'+str(ind)] = pd.concat([IMF_i[IMF_name], self.dataframe.get(self.cols[1:])],axis=1)


    def Normalization(self, **EMD_para):
        '''
        call function scale_EMD(), decide whether use EMD to do preprocessing
        normalize the training data and appy the same scale to validation and test data
        '''
        
        i_split1 = int(len(self.dataframe) * self.split1)
        i_split2 = int(len(self.dataframe) * self.split2)
        
        if len(EMD_para) ==0:
            self.scale_EMD()
            
            self.data_train_original = self.dataframe.get(self.cols)[:i_split1]
            self.data_val_original   = self.dataframe.get(self.cols)[i_split1:i_split2]
            self.data_test_original  = self.dataframe.get(self.cols)[i_split2:]
            
        else:
            self.scale_EMD(activate_EMD=True)
            IMF_number = EMD_para['IMF_num']
            
            print('processing the data of IM'+ str(IMF_number))
            
            if IMF_number in range(self.IMFs.shape[0]):
                self.data_train_original = self.df_names_IMF['IMF'+str(IMF_number)][:i_split1]
                self.data_val_original   = self.df_names_IMF['IMF'+str(IMF_number)][i_split1:i_split2]
                self.data_test_original  = self.df_names_IMF['IMF'+str(IMF_number)][i_split2:]
            else:
                print("Oops!IMF_number was no valid number. it must between 0 and "+str(self.IMFs.shape[0]-1))

        self.min_max_scaler = preprocessing.MinMaxScaler().fit(self.data_train_original.values)

        self.data_train = self.min_max_scaler.transform(self.data_train_original.values)
        self.data_val = self.min_max_scaler.transform(self.data_val_original.values)
        self.data_test = self.min_max_scaler.transform(self.data_test_original.values)

        self.len_train  = len(self.data_train_original)
        self.len_val    = len(self.data_val_original)
        self.len_test   = len(self.data_test_original)
    
    def get_pre_time(self):
        data_windows = []

        for i in range((self.len_test-self.input_timesteps)//self.pre_len):
            data_windows.append(self.data_test_original.index[i*self.pre_len:i*self.pre_len+self.seq_len])
            
        pre_time = np.array([p[self.input_timesteps:] for p in data_windows])
       
        return pre_time
    
    def get_test_data(self):
        '''
        Create x, y test data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise reduce size of the training split.
        '''
        data_windows = []

        for i in range((self.len_test-self.input_timesteps)//self.pre_len):
            data_windows.append(self.data_test[i*self.pre_len:i*self.pre_len+self.seq_len])

        x = np.array([p[:self.input_timesteps,:] for p in data_windows])
        y = np.array([p[self.input_timesteps:,0] for p in data_windows])
        return x,y
    
    def get_train_data(self):
        '''
        Create x, y train data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise use generate_training_window() method.
        '''
        train_x = []
        train_y = []
        for i in range(self.len_train-self.seq_len):
            data_window = self.data_train[i:i+self.seq_len]
            train_x.append(data_window[:self.input_timesteps,:])
            train_y.append(data_window[self.input_timesteps:,0])
            
        train_x = np.array(train_x)
        train_y = np.array(train_y)
        
        sfl = list(range(len(train_x)))
        random.shuffle(sfl)
        train_x = train_x[sfl]
        train_y = train_y[sfl]
        
        return train_x, train_y
            
    def get_val_data(self):
        val_x = []
        val_y = []
        for i in range(self.len_val-self.seq_len):
            data_window = self.data_val[i:i+self.seq_len]
            val_x.append(data_window[:self.input_timesteps,:])
            val_y.append(data_window[self.input_timesteps:,0])
        val_x = np.array(val_x)
        val_y = np.array(val_y)
        
        sfl = list(range(len(val_x)))
        random.shuffle(sfl)
        val_x = val_x[sfl]
        val_y = val_y[sfl]
        
        return val_x, val_y
    
    def training_batch_generator(self, batch_size):
        train_x, train_y = self.get_train_data()
        return DataLoaderSequence(train_x,train_y, batch_size)
    
    def val_batch_generator(self, batch_size):
        val_x,val_y = self.get_val_data()
        return DataLoaderSequence(val_x, val_y, batch_size)

In [360]:
# configs = json.load(open('config.json', 'r'))
# data = DataLoader(
#     filename=os.path.join('data', configs['data']['filename']),
#     split1=configs['data']['train_test_split1'],
#     split2=configs['data']['train_test_split2'],
#     cols=configs['data']['columns'],
#     pre_len=configs['model']['layers'][4]['neurons'],
#     input_timesteps=configs['model']['layers'][0]['input_timesteps'],
#     seq_len = configs['data']['sequence_length'],
#     IMF_num = 10
# )

the input cols are: ['Consumption', 'hour', 'holiday_indicator']
scaling Consumption is done!
the signal is decomposed into 11 parts
processing the data of IM10


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [361]:
# data.dataframe